### Imports

In [2]:
import pandas as pd 
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import json
import json_repair 
from tqdm.auto import tqdm
import time

load_dotenv()

True

### Test the model

In [3]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    temperature=0,
    max_retries=2,
)


prompt_template = ChatPromptTemplate([
    ("system","""
        you are a economic expert, you will receive some of category about my distribution of my salary .
        for example, my salary is 5000 Egyptian pound, I will suspense in the following(in pound):
        electricity bill --> 200 
        water bill --> 400
        internet bill --> 200
        food --> 1000
        shopping --> 900
        etc...
        Your role is to make optimization for my suspension to improve saving from my salary
     """),
    ("human","""
        my salary is 10000 Egyptian pound:
        electricity bill --> 300 
        internet --> 200
        food --> 2000
        shopping --> 3000
        gas --> 350 
        water bill --> 290
     """)
])




chain = prompt_template | model | StrOutputParser()

result= chain.invoke({"topic":"lawyers", "joke_count":5})

print(result)

Okay, let's analyze your current spending and see where we can optimize to improve your savings.

**Current Situation:**

*   **Salary:** 10,000 EGP
*   **Expenses:**
    *   Electricity: 300 EGP
    *   Internet: 200 EGP
    *   Food: 2000 EGP
    *   Shopping: 3000 EGP
    *   Gas: 350 EGP
    *   Water: 290 EGP
*   **Total Expenses:** 6140 EGP
*   **Savings:** 10,000 - 6140 = 3860 EGP

**Analysis and Recommendations:**

You're currently saving a decent portion of your salary (38.6%), which is a good starting point. However, we can likely improve this further by focusing on the areas where you're spending the most.

Here's a breakdown with specific recommendations:

1.  **Shopping (3000 EGP - Highest Priority for Optimization):**

    *   **Problem:** This is the largest expense category and a prime target for savings. "Shopping" is broad, so we need to understand what it includes. Is it clothes, entertainment, personal care, gifts, or a combination?
    *   **Recommendations:**
    

### parse the JSON output

In [5]:
def parse_json(text):
    try:
        return json_repair.loads(text)
    except:
        return None

### Schematic of Personal Data

In [ ]:
def prompt_template_generation(data):

    # 2. Define expected JSON output schema
    response_schemas = [
    ResponseSchema(
        name="input_summary",
        description="A brief summary of the user's original inputted income and expense data"
    ),
    ResponseSchema(
        name="recommended_savings",
        description="The suggested amount to save each month, based on the user's income and spending, in the specified currency (e.g., EGP)"
    ),
    ResponseSchema(
        name="optimized_budget",
        description="A dictionary showing the recommended monthly spending for each category after optimization"
    ),
    ResponseSchema(
        name="financial_tips",
        description="Concise, practical tips to help the user better manage their budget and increase savings"
    )
]

    parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = parser.get_format_instructions()

    # 3. Build the prompt
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", "You are a financial assistant that helps users optimize their monthly budgets."),
        ("human", """Here is the user's budget data: {budget_data}

    {format_instructions}""")
    ])

    # 4. Combine into a Runnable chain
    chain = (
        prompt_template |
        model |
        parser
    )

    # 5. Example Input
    input_data = {
        "currency_type": "EGP",
        "budget_data": str(data.to_dict()),
        "format_instructions": format_instructions
    }

    # 6. Run the chain
    try:
        result = chain.invoke(input_data)
    except:
        time.sleep(20)
    # 7. Print final JSON result
    # str(result)
    repaired_json = json_repair.repair_json(str(result))
    # repaired_json
    return repaired_json

In [2]:
df = pd.read_csv("generated_data.csv")
df

,Date,Month,Day,Year,Income,Electricity,Gas,Internet,Water,Food,...,Investment,Emergency Fund,Retirement,Education,Travel,Gifts,Home Maintenance,Others,Total_Expenses,Savings_Rate
0,2023-11-24,11,24,2023,2533.60,87.73,44.90,51.35,33.64,438.51,...,273.38,229.54,441.41,129.58,290.71,45.56,88.61,110.54,4426.85,-0.747257
1,2023-02-22,2,22,2023,2511.30,67.09,56.95,62.49,44.91,436.76,...,274.73,166.38,357.25,126.06,273.36,52.71,94.59,94.81,4469.68,-0.779827
2,2023-04-18,4,18,2023,3204.21,86.46,53.67,49.71,42.16,405.46,...,288.14,165.30,466.10,154.03,326.15,44.25,99.97,115.39,4446.90,-0.387830
3,2023-11-26,11,26,2023,2950.62,72.48,44.93,61.47,36.20,413.53,...,247.62,190.53,479.38,151.75,356.53,57.22,80.46,108.83,4456.80,-0.510462
4,2023-12-16,12,16,2023,3461.75,88.03,55.37,56.16,36.70,345.31,...,284.87,172.92,472.56,175.35,350.22,51.98,99.54,84.47,4485.88,-0.295842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2023-07-29,7,29,2023,2520.98,73.21,47.33,54.54,45.68,458.02,...,311.08,188.88,387.38,142.92,256.58,52.13,118.84,97.57,4426.56,-0.755889
2996,2023-04-20,4,20,2023,3593.53,80.21,46.43,63.51,36.71,437.46,...,348.99,231.92,434.02,152.46,333.64,50.27,118.44,117.65,4599.60,-0.279967
2997,2023-05-19,5,19,2023,3408.05,67.14,51.62,50.90,37.85,432.35,...,250.62,233.28,404.04,128.47,288.56,45.17,112.21,112.56,4452.42,-0.306442
2998,2023-10-09,10,9,2023,2463.81,93.39,54.22,68.77,44.79,381.15,...,250.92,221.41,364.54,121.88,338.84,48.65,91.29,91.88,4233.24,-0.718168


In [3]:
modi_df = df.drop(columns=['Date','Month','Day','Year','Total_Expenses','Savings_Rate'])
modi_df.columns

Index(['Income', 'Electricity', 'Gas', 'Internet', 'Water', 'Food',
       'Groceries', 'Rent', 'Transportation', 'Healthcare', 'Shopping',
       'Entertainment', 'Dining Out', 'Subscription Services', 'Clothing',
       'Electronics', 'Zakat', 'Investment', 'Emergency Fund', 'Retirement',
       'Education', 'Travel', 'Gifts', 'Home Maintenance', 'Others'],
      dtype='object')

In [ ]:
test = modi_df.loc[200]
output = prompt_template_generation(test)
# output
json.loads(output)
# parse_json(output)

{'input_summary': 'The user has a monthly income of 2932.38. Their expenses cover a wide range of categories including utilities, housing, food, transportation, healthcare, shopping, entertainment, savings, and miscellaneous spending.', 'recommended_savings': "Based on your income and expenses, it's recommended to aim for a savings rate of at least 15% of your income. This would be approximately 439.86 per month, in addition to your current savings and investments.", 'optimized_budget': {'Income': 2932.38, 'Electricity': 65.0, 'Gas': 45.0, 'Internet': 60.0, 'Water': 30.0, 'Food': 300.0, 'Groceries': 270.0, 'Rent': 697.27, 'Transportation': 140.0, 'Healthcare': 100.0, 'Shopping': 200.0, 'Entertainment': 90.0, 'Dining Out': 140.0, 'Subscription Services': 25.0, 'Clothing': 100.0, 'Electronics': 50.0, 'Zakat': 213.67, 'Investment': 350.0, 'Emergency Fund': 250.0, 'Retirement': 450.0, 'Education': 130.0, 'Travel': 200.0, 'Gifts': 50.0, 'Home Maintenance': 100.0, 'Others': 80.0}, 'financial

{'input_summary': 'The user has a monthly income of 2932.38. Their expenses cover a wide range of categories including utilities, housing, food, transportation, healthcare, shopping, entertainment, savings, and miscellaneous spending.',
 'recommended_savings': "Based on your income and expenses, it's recommended to aim for a savings rate of at least 15% of your income. This would be approximately 439.86 per month, in addition to your current savings and investments.",
 'optimized_budget': {'Income': 2932.38,
  'Electricity': 65.0,
  'Gas': 45.0,
  'Internet': 60.0,
  'Water': 30.0,
  'Food': 300.0,
  'Groceries': 270.0,
  'Rent': 697.27,
  'Transportation': 140.0,
  'Healthcare': 100.0,
  'Shopping': 200.0,
  'Entertainment': 90.0,
  'Dining Out': 140.0,
  'Subscription Services': 25.0,
  'Clothing': 100.0,
  'Electronics': 50.0,
  'Zakat': 213.67,
  'Investment': 350.0,
  'Emergency Fund': 250.0,
  'Retirement': 450.0,
  'Education': 130.0,
  'Travel': 200.0,
  'Gifts': 50.0,
  'Home 

In [ ]:
# make json file 
# with open("dataset.jsonl", 'w') as f:
#         json.dump([], f)
# append the LLM output into the file 
itarate = modi_df.shape[0]
for i in tqdm(range(itarate)):
    test = modi_df.loc[i]
    try:
        output = prompt_template_generation(test)
        itarate = i
    except UnboundLocalError:
        time.sleep(30)
        continue
    output = json.loads(output)
    with open("dataset.jsonl", 'a') as f:
        f.write(json.dumps(output) + '\n')

    

  0%|          | 0/3000 [00:00<?, ?it/s]

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 29
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 34
}
].
Retrying langchain_google_gena

KeyboardInterrupt: 

In [7]:
modi_df.loc[2932].to_dict()

{'Income': 3271.32,
 'Electricity': 68.8,
 'Gas': 44.46,
 'Internet': 63.46,
 'Water': 43.37,
 'Food': 387.04,
 'Groceries': 304.22,
 'Rent': 861.92,
 'Transportation': 159.58,
 'Healthcare': 99.91,
 'Shopping': 173.13,
 'Entertainment': 102.72,
 'Dining Out': 153.67,
 'Subscription Services': 31.06,
 'Clothing': 95.52,
 'Electronics': 52.21,
 'Zakat': 222.91,
 'Investment': 359.92,
 'Emergency Fund': 229.5,
 'Retirement': 415.64,
 'Education': 149.66,
 'Travel': 305.61,
 'Gifts': 52.39,
 'Home Maintenance': 118.51,
 'Others': 89.79}

In [6]:
import json

input_file = 'dataset.jsonl'
output_file = 'updated_dataset.jsonl'

# Assuming modi_df is your pandas DataFrame with 1366 rows

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for i, line in enumerate(infile):
        if i >= len(modi_df):  # Safety check to prevent index errors
            break
            
        data = json.loads(line)
        # Update with the corresponding row from modi_df
        row_data = modi_df.loc[i].to_dict()
        data["input_summary"] = str(row_data)  # Or format this as you prefer
        
        # Write the modified data
        outfile.write(json.dumps(data) + '\n')

print(f"File processed. Updated data written to {output_file}")

File processed. Updated data written to updated_dataset.jsonl


In [4]:
df = pd.read_csv("business_financial_data.csv")
df

,Date,Month,Year,Total_Revenue,Employee_Salaries,Loan_Payment,Raw_Materials,Utilities,Rent,Marketing,...,Maintenance,Taxes,Transfer_Money,Profits,Losses,Total_Expenses,Profit_Margin,Expense_Ratio,Debt_Coverage_Ratio,Employee_Cost_Ratio
0,1960-01-01,1,1960,520914.02,128625.48,23312.72,91696.32,8567.53,21060.20,33529.62,...,8589.39,45779.57,40064.26,141729.09,0.0,379184.93,0.272078,0.727922,6.079475,0.246923
1,1960-02-01,2,1960,522482.67,152022.37,22903.30,102677.97,8742.63,17038.99,32752.37,...,8966.44,56858.20,38039.13,99868.39,0.0,422614.28,0.191142,0.808858,4.360437,0.290962
2,1960-03-01,3,1960,552124.15,154667.67,27303.46,106891.95,8086.95,22838.69,28906.81,...,10355.56,55425.60,40928.23,116919.17,0.0,435204.98,0.211762,0.788238,4.282211,0.280132
3,1960-04-01,4,1960,459184.74,140522.46,21848.44,91983.73,7042.40,18667.84,31221.16,...,9128.52,46504.67,45239.86,73068.51,0.0,386116.23,0.159127,0.840873,3.344335,0.306026
4,1960-05-01,5,1960,450670.80,160310.71,22475.52,96383.66,9174.86,20840.00,30512.55,...,10828.00,45935.72,34385.20,32864.74,0.0,417806.06,0.072924,0.927076,1.462246,0.355716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,2024-08-01,8,2024,569864.84,162002.52,27165.53,91525.78,7698.87,17487.65,32018.49,...,10426.95,56022.19,36293.03,146150.61,0.0,423714.23,0.256465,0.743535,5.380002,0.284282
776,2024-09-01,9,2024,433000.30,159266.13,22585.03,89212.78,7557.30,18355.76,27194.21,...,9384.80,52963.37,37174.54,25239.19,0.0,407761.11,0.058289,0.941711,1.117519,0.367820
777,2024-10-01,10,2024,477328.88,156382.17,24622.55,103930.79,8796.15,17220.39,33012.52,...,9131.99,53032.81,38163.06,51657.03,0.0,425671.85,0.108221,0.891779,2.097956,0.327619
778,2024-11-01,11,2024,456117.04,158984.42,25028.24,109895.73,8706.31,17544.55,32460.68,...,8594.34,44229.47,41899.99,31809.81,0.0,424307.23,0.069740,0.930260,1.270957,0.348561


In [5]:
modi_df = df.drop(columns=['Employee_Cost_Ratio','Debt_Coverage_Ratio','Expense_Ratio','Profit_Margin','Total_Expenses','Losses'])
modi_df.columns

Index(['Date', 'Month', 'Year', 'Total_Revenue', 'Employee_Salaries',
       'Loan_Payment', 'Raw_Materials', 'Utilities', 'Rent', 'Marketing',
       'Insurance', 'Office_Supplies', 'Maintenance', 'Taxes',
       'Transfer_Money', 'Profits'],
      dtype='object')

In [6]:
def business_prompt_template_generation(data):
    # Define expected JSON output schema for business financial analysis
    response_schemas = [
        ResponseSchema(
            name="business_financial_summary",
            description="A comprehensive summary of the company's financial data including revenue, expenses, profits/losses, and key metrics"
        ),
        ResponseSchema(
            name="profitability_analysis",
            description="Analysis of the company's profitability including profit margins, revenue growth, and expense management"
        ),
        ResponseSchema(
            name="cash_flow_recommendations",
            description="Recommended strategies for improving cash flow management and liquidity"
        ),
        ResponseSchema(
            name="expense_optimization",
            description="A dictionary showing recommended adjustments to expense categories to improve profitability"
        ),
        ResponseSchema(
            name="investment_opportunities",
            description="Potential areas for business investment or expansion based on financial performance"
        ),
        ResponseSchema(
            name="risk_assessment",
            description="Identification of financial risks and recommended mitigation strategies"
        )
    ]

    parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = parser.get_format_instructions()

    # Build the prompt for business financial analysis
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", """You are a sophisticated business financial analyst specializing in corporate financial management. 
        Provide detailed insights and strategic recommendations based on company financial data.
        Focus on profitability, cash flow optimization, expense management, and growth opportunities.
        Your analysis should be data-driven, actionable, and specific to the business context."""),
        ("human", """Here is the company's financial data: {financial_data}

        Please provide a comprehensive financial analysis with actionable recommendations.
        
        {format_instructions}""")
    ])

    # Combine into a Runnable chain
    chain = (
        prompt_template |
        model |
        parser
    )

    # Input data preparation
    input_data = {
        "financial_data": str(data.to_dict()),
        "format_instructions": format_instructions
    }

    # Run the chain with error handling
    try:
        result = chain.invoke(input_data)
    except Exception as e:
        print(f"Error executing chain: {e}")
        time.sleep(20)
        try:
            # Retry once after delay
            result = chain.invoke(input_data)
        except Exception as e:
            print(f"Second attempt failed: {e}")
            return {"error": "Failed to process financial data after multiple attempts"}

    # Process and return result
    try:
        repaired_json = json_repair.repair_json(str(result))
        return repaired_json
    except Exception as e:
        print(f"Error repairing JSON: {e}")
        return {"error": "Failed to repair JSON output", "raw_result": str(result)}

In [6]:
modi_df

,Date,Month,Year,Total_Revenue,Employee_Salaries,Loan_Payment,Raw_Materials,Utilities,Rent,Marketing,Insurance,Office_Supplies,Maintenance,Taxes,Transfer_Money,Profits
0,1960-01-01,1,1960,520914.02,128625.48,23312.72,91696.32,8567.53,21060.20,33529.62,13141.22,4882.88,8589.39,45779.57,40064.26,141729.09
1,1960-02-01,2,1960,522482.67,152022.37,22903.30,102677.97,8742.63,17038.99,32752.37,15891.63,4760.38,8966.44,56858.20,38039.13,99868.39
2,1960-03-01,3,1960,552124.15,154667.67,27303.46,106891.95,8086.95,22838.69,28906.81,15234.18,5494.11,10355.56,55425.60,40928.23,116919.17
3,1960-04-01,4,1960,459184.74,140522.46,21848.44,91983.73,7042.40,18667.84,31221.16,14391.74,4805.27,9128.52,46504.67,45239.86,73068.51
4,1960-05-01,5,1960,450670.80,160310.71,22475.52,96383.66,9174.86,20840.00,30512.55,15830.76,5514.28,10828.00,45935.72,34385.20,32864.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,2024-08-01,8,2024,569864.84,162002.52,27165.53,91525.78,7698.87,17487.65,32018.49,13860.29,5505.96,10426.95,56022.19,36293.03,146150.61
776,2024-09-01,9,2024,433000.30,159266.13,22585.03,89212.78,7557.30,18355.76,27194.21,16726.06,4515.67,9384.80,52963.37,37174.54,25239.19
777,2024-10-01,10,2024,477328.88,156382.17,24622.55,103930.79,8796.15,17220.39,33012.52,14485.56,5056.92,9131.99,53032.81,38163.06,51657.03
778,2024-11-01,11,2024,456117.04,158984.42,25028.24,109895.73,8706.31,17544.55,32460.68,13911.96,4951.53,8594.34,44229.47,41899.99,31809.81


In [13]:
itarate = modi_df.shape[0]
for i in tqdm(range(itarate)):
    test = modi_df.loc[i]
    try:
        output = business_prompt_template_generation(test)
        itarate = i
    except UnboundLocalError:
        time.sleep(61)
        continue
    output = json.loads(output)
    with open("business_dataset.jsonl", 'a') as f:
        f.write(json.dumps(output) + '\n')

    

  0%|          | 0/780 [00:00<?, ?it/s]

In [7]:
modi_df.loc[700].to_dict()

{'Date': '2018-05-01',
 'Month': 5,
 'Year': 2018,
 'Total_Revenue': 526352.11,
 'Employee_Salaries': 165851.51,
 'Loan_Payment': 23506.69,
 'Raw_Materials': 94326.72,
 'Utilities': 9173.98,
 'Rent': 17038.7,
 'Marketing': 29864.6,
 'Insurance': 15844.99,
 'Office_Supplies': 4422.06,
 'Maintenance': 8500.11,
 'Taxes': 51556.86,
 'Transfer_Money': 35452.41,
 'Profits': 106265.88999999996}